<a href="https://colab.research.google.com/github/YawManuel/Bosea/blob/main/Barbados_Land_Survey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

yawmanuel_barbados_path = kagglehub.dataset_download('yawmanuel/barbados')
yawmanuel_barbados_testfile_path = kagglehub.dataset_download('yawmanuel/barbados-testfile')
yawmanuel_survey_plans_path = kagglehub.dataset_download('yawmanuel/survey-plans')

print('Data source import complete.')


In [ ]:
# Kaggle/Local Notebook: Land Survey Plot Extraction and Metadata Pipeline (Path-Fixed)
# =====================================================================
# Set BASE_DIR to your data root (e.g., '/kaggle/input/dataset/' or './data/').
# Assumes: train.csv, test.csv, survey-images/ folder all in BASE_DIR.

# Cell 1: Install Dependencies (Run once)
import sys
!{sys.executable} -m pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu
!{sys.executable} -m pip install opencv-python pytesseract geopandas shapely scikit-learn albumentations pandas numpy matplotlib seaborn detectron2 -f https://dl.fbaipublicfiles.com/detectron2/wheels/cpu/torch1.13/index.html
!apt-get update && apt-get install -y tesseract-ocr  # For OCR; skip if local

Looking in indexes: https://download.pytorch.org/whl/cpu


In [ ]:
# Cell 2: Imports
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import re
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
from sklearn.metrics import mean_squared_error, accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.base import BaseEstimator, RegressorMixin, ClassifierMixin
import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import models, transforms
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2
import pytesseract
from shapely.wkt import loads
from shapely.geometry import Polygon
import geopandas as gpd
from difflib import SequenceMatcher
import warnings
warnings.filterwarnings('ignore')

# Set device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# PATH CONFIGURATION
BASE_DIR = './'  # CHANGE THIS: e.g., '/kaggle/input/your-dataset/' or './'
TRAIN_CSV = os.path.join(BASE_DIR, '/kaggle/input/barbados/Train.csv')
TEST_CSV = os.path.join(BASE_DIR, '/kaggle/input/barbados-testfile/Test.csv')
IMAGE_DIR = os.path.join(BASE_DIR, '/kaggle/input/survey-plans')  # Folder with {ID}.jpg

print(f"Train path: {TRAIN_CSV} (exists: {os.path.exists(TRAIN_CSV)})")
print(f"Test path: {TEST_CSV} (exists: {os.path.exists(TEST_CSV)})")
print(f"Images dir: {IMAGE_DIR} (exists: {os.path.exists(IMAGE_DIR)})")

In [ ]:
# Cell 3: Helper Functions (Unchanged from previous)
def clean_target_survey(text: str) -> str:
    text = text.lower()
    text = re.sub(r"[.,]", " ", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()

def format_dataset(df: pd.DataFrame) -> pd.DataFrame:
    df["TargetSurvey"] = (
        df["Land Surveyor"].astype(str).str.strip() + " " +
        df["Surveyed For"].astype(str).str.strip() + " " +
        df["Address"].astype(str).str.strip()
    ).apply(clean_target_survey)
    columns_to_keep = ['ID', 'TargetSurvey', 'Certified date', 'Total Area', 'Unit of Measurement', 'Parish', 'LT Num', 'geometry']
    return df[columns_to_keep]

def polygon_to_mask(polygon, width=256, height=256):
    if polygon is None or polygon.is_empty:
        return np.zeros((height, width), dtype=np.uint8)
    minx, miny, maxx, maxy = polygon.bounds
    coords = np.array(polygon.exterior.coords)
    x_scale = width / (maxx - minx) if maxx != minx else 1
    y_scale = height / (maxy - miny) if maxy != miny else 1
    pixel_coords = np.array([[(x - minx) * x_scale, (maxy - y) * y_scale] for x, y in coords], dtype=np.int32)
    mask = np.zeros((height, width), dtype=np.uint8)
    cv2.fillPoly(mask, [pixel_coords], 1)
    return mask

def compute_iou(mask1, mask2):
    intersection = np.logical_and(mask1, mask2).sum()
    union = np.logical_or(mask1, mask2).sum()
    return intersection / union if union > 0 else 0

def wer(pred_text, gt_text):
    return 1 - SequenceMatcher(None, pred_text, gt_text).ratio()

def ocr_extract_metadata(image_path, id_key):
    if not os.path.exists(image_path):
        # Mock defaults if no image
        return {'ID': id_key, 'Land Surveyor': 'Unknown', 'Surveyed For': 'Unknown',
                'Certified date': 'Unknown', 'Total Area': 0.0, 'Unit of Measurement': 'sq m',
                'Address': 'Unknown', 'Parish': 'St. Philip', 'LT Num': 'Unknown',
                'geometry': 'POLYGON Z ((0 0 0, 0 0 0, 0 0 0, 0 0 0))'}

    img = cv2.imread(image_path, 0)
    if img is None:
        return ocr_extract_metadata(image_path, id_key)  # Retry mock
    img = cv2.medianBlur(img, 5)
    img = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]
    text = pytesseract.image_to_string(img)

    data = {'ID': id_key}
    # Tuned regex (as before)
    surveyor_match = re.search(r'(?:Land Surveyor|Surveyor)\s*[:\s]*(.+?)(?:\n|$)', text, re.IGNORECASE | re.DOTALL)
    data['Land Surveyor'] = surveyor_match.group(1).strip() if surveyor_match else 'Unknown'

    surveyed_match = re.search(r'surveyed\s+for\s+(.+?)(?:\n|$)', text, re.IGNORECASE | re.DOTALL)
    data['Surveyed For'] = surveyed_match.group(1).strip() if surveyed_match else 'Unknown'

    date_match = re.search(r'certified?\s+(.+?)(?:\d{4}|\n)', text, re.IGNORECASE)
    data['Certified date'] = date_match.group(1).strip() if date_match else 'Unknown'

    area_match = re.search(r'total\s*(?:area|land)\s*[:\s]*(\d+(?:\.\d+)?)\s*(sq\s*m|sqm)', text, re.IGNORECASE)
    data['Total Area'] = float(area_match.group(1)) if area_match else 0.0
    data['Unit of Measurement'] = area_match.group(2).strip() if area_match else 'sq m'

    address_match = re.search(r'lot\s+\d+\s*,?\s*(.+?)(?:,\s*stage|\n)', text, re.IGNORECASE | re.DOTALL)
    data['Address'] = address_match.group(1).strip() if address_match else 'Unknown'

    parish_match = re.search(r'st\.\s*philip', text, re.IGNORECASE)
    data['Parish'] = parish_match.group(0).strip() if parish_match else 'St. Philip'

    lt_match = re.search(r'(?:lt\s+num|land\s+tax\s+ref|file\s+no)[:\s]*([^\n]+)', text, re.IGNORECASE)
    data['LT Num'] = lt_match.group(1).strip() if lt_match else 'Unknown'

    # Polygon from contours
    contours, _ = cv2.findContours(img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    if contours:
        largest = max(contours, key=cv2.contourArea)
        epsilon = 0.02 * cv2.arcLength(largest, True)
        approx = cv2.approxPolyDP(largest, epsilon, True)
        if len(approx) >= 3:
            coords = [(pt[0][0], pt[0][1]) for pt in approx]
            wkt_coords = ', '.join([f'{x} {y} 0' for x, y in coords]) + ', 0 0 0'
            data['geometry'] = f"POLYGON Z (({wkt_coords}))"
        else:
            data['geometry'] = 'POLYGON Z ((0 0 0, 0 0 0, 0 0 0, 0 0 0))'
    else:
        data['geometry'] = 'POLYGON Z ((0 0 0, 0 0 0, 0 0 0, 0 0 0))'

    return data

In [ ]:
# Cell 4: Load & Preprocess Train (for Model Training)
df_train = pd.read_csv(TRAIN_CSV)
extracted_train = []
for idx, row in df_train.iterrows():
    id_key = row['ID']
    image_path = os.path.join(IMAGE_DIR, f"{id_key}.jpg")
    extracted = ocr_extract_metadata(image_path, id_key)
    # Merge with CSV (prioritize OCR)
    for key in extracted:
        if key != 'ID' and pd.isna(row.get(key, np.nan)):
            row[key] = extracted[key]
    extracted_train.append(row)
df_train = pd.DataFrame(extracted_train)
df_train = format_dataset(df_train)
print(f"Train DF: {df_train.shape}")

# Cell 5: EDA (Optional for Train)
def eda(df):
    print("Shape:", df.shape)
    print("Missing:\n", df.isnull().sum())
    df['Total Area'].hist(bins=20)
    plt.title('Total Area Dist')
    plt.show()
    df['polygon'] = df['geometry'].apply(lambda wkt: loads(wkt) if wkt else None)
    print("Valid Geoms:", df['polygon'].apply(lambda p: p.is_valid if p else False).mean())
eda(df_train)


In [ ]:
# Cell 6: Preprocess Train
def preprocess_data(df):
    df = df.dropna(subset=['Total Area', 'geometry'])
    le_parish = LabelEncoder()
    df['Parish Encoded'] = le_parish.fit_transform(df['Parish'])
    scaler = StandardScaler()
    df['Total Area Normalized'] = scaler.fit_transform(df[['Total Area']]).flatten()
    df['polygon'] = df['geometry'].apply(lambda wkt: loads(wkt) if wkt else None)
    df['computed_area'] = df['polygon'].apply(lambda p: p.area if p else 0)
    df['perimeter'] = df['polygon'].apply(lambda p: p.length if p else 0)
    df['centroid_x'] = df['polygon'].apply(lambda p: p.centroid.x if p else 0)
    df['centroid_y'] = df['polygon'].apply(lambda p: p.centroid.y if p else 0)
    train_df, temp_df = train_test_split(df, test_size=0.3, random_state=42)
    val_df, test_df = train_test_split(temp_df, test_size=0.5, random_state=42)
    return train_df, val_df, test_df, scaler, le_parish

train_df, val_df, test_df, scaler, le_parish = preprocess_data(df_train)

# Cell 7: Datasets (Unchanged)
class SurveyDataset(Dataset):
    def __init__(self, df, features, image_dir=None):
        self.df = df
        self.features = df[features].values
        self.labels = df['Total Area'].values
        self.image_dir = image_dir

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        feat = torch.tensor(self.features[idx], dtype=torch.float32)
        label = torch.tensor(self.labels[idx], dtype=torch.float32)
        id_key = self.df.iloc[idx]['ID']
        img_path = os.path.join(self.image_dir, f"{id_key}.jpg") if self.image_dir else None
        if img_path and os.path.exists(img_path):
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            transform = A.Compose([A.Resize(224, 224), A.Normalize(), ToTensorV2()])
            img = transform(image=img)['image']
        else:
            img = torch.zeros(3, 224, 224)
        return {'features': feat, 'image': img, 'label': label}

    def collate_fn(self, batch):
        features = torch.stack([item['features'] for item in batch])
        images = torch.stack([item['image'] for item in batch])
        labels = torch.stack([item['label'] for item in batch])
        return {'features': features, 'images': images, 'labels': labels}

features = ['Total Area Normalized', 'perimeter', 'centroid_x', 'centroid_y', 'Parish Encoded']
train_dataset = SurveyDataset(train_df, features, IMAGE_DIR)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, collate_fn=train_dataset.collate_fn)

# Cell 8: Models (Unchanged)
class StackingRegressor(BaseEstimator, RegressorMixin):
    def __init__(self):
        self.rf = RandomForestRegressor(n_estimators=100, random_state=42)
        self.svm = SVR(kernel='rbf')
        self.meta = RandomForestRegressor(n_estimators=50, random_state=42)

    def fit(self, X, y):
        self.rf.fit(X, y)
        self.svm.fit(X, y)
        meta_X = np.column_stack((self.rf.predict(X), self.svm.predict(X)))
        self.meta.fit(meta_X, y)
        return self

    def predict(self, X):
        rf_pred = self.rf.predict(X)
        svm_pred = self.svm.predict(X)
        meta_X = np.column_stack((rf_pred, svm_pred))
        return self.meta.predict(meta_X)

class MultiModalModel(torch.nn.Module):
    def __init__(self, num_features):
        super().__init__()
        self.cnn = models.resnet18(pretrained=True)
        self.cnn.fc = torch.nn.Linear(self.cnn.fc.in_features, 128)
        self.fc = torch.nn.Linear(128 + num_features, 1)

    def forward(self, images, features):
        img_feat = self.cnn(images)
        combined = torch.cat((img_feat, features), dim=1)
        return self.fc(combined)

stack_model = StackingRegressor()
multi_model = MultiModalModel(len(features)).to(device)
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(multi_model.parameters(), lr=0.001)

# Cell 9: Train Models (on Train)
X_train = train_df[features].values
y_train = train_df['Total Area'].values
stack_model.fit(X_train, y_train)

multi_model.train()
for epoch in range(5):  # Reduced for speed
    for batch in train_loader:
        images = batch['images'].to(device)
        feats = batch['features'].to(device)
        labels = batch['labels'].to(device).unsqueeze(1)
        optimizer.zero_grad()
        outputs = multi_model(images, feats)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
    if epoch % 2 == 0:
        print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")

torch.save(multi_model.state_dict(), 'multi_model.pth')

In [ ]:
# Cell 10: Evaluation (Optional, on Val/Test Split)
val_dataset = SurveyDataset(val_df, features, IMAGE_DIR)
val_loader = DataLoader(val_dataset, batch_size=8, collate_fn=val_dataset.collate_fn)

def evaluate(df_val, stack_model, multi_model, val_loader):
    X_val = df_val[features].values
    y_val = df_val['Total Area'].values
    stack_pred = stack_model.predict(X_val)
    stack_rmse = np.sqrt(mean_squared_error(y_val, stack_pred))

    multi_preds = []
    multi_model.eval()
    with torch.no_grad():
        for batch in val_loader:
            images = batch['images'].to(device)
            feats = batch['features'].to(device)
            outputs = multi_model(images, feats)
            multi_preds.extend(outputs.cpu().numpy().flatten())
    multi_rmse = np.sqrt(mean_squared_error(y_val[:len(multi_preds)], multi_preds)) if multi_preds else 0

    # Other metrics (mock for test; adapt for val)
    avg_wer = 0.1  # Mock
    mca = 0.9  # Mock
    ious = [0.85] * len(df_val)  # Mock
    avg_iou = np.mean(ious)

    print(f"Stack RMSE: {stack_rmse:.2f} | Multi RMSE: {multi_rmse:.2f} | WER: {avg_wer:.2f} | MCA: {mca:.2f} | IoU: {avg_iou:.2f}")
    return {'stack_rmse': stack_rmse, 'wer': avg_wer, 'mca': mca, 'iou': avg_iou}

if len(val_df) > 0:
    results = evaluate(val_df, stack_model, multi_model, val_loader)



In [ ]:
# Cell 11: Inference Function (Updated for Test)
def inference(test_df, stack_model, multi_model, image_dir=IMAGE_DIR, output_csv='test_predictions.csv', output_shp='predicted_test_shapefile.shp'):
    # Enhance with OCR for each ID
    extracted_test = []
    for idx, row in test_df.iterrows():
        id_key = row['ID']
        image_path = os.path.join(image_dir, f"{id_key}.jpg")
        extracted = ocr_extract_metadata(image_path, id_key)
        # Merge (test has only ID, so use extracted)
        row = pd.Series(extracted)
        extracted_test.append(row)

    enhanced_df = pd.DataFrame(extracted_test)
    enhanced_df = format_dataset(enhanced_df)

    # Preprocess for prediction
    enhanced_df['Parish Encoded'] = le_parish.transform(enhanced_df['Parish'])
    enhanced_df['Total Area Normalized'] = scaler.transform(enhanced_df[['Total Area']])[:, 0]  # Use extracted if available
    enhanced_df['polygon'] = enhanced_df['geometry'].apply(lambda wkt: loads(wkt) if wkt else None)
    enhanced_df['computed_area'] = enhanced_df['polygon'].apply(lambda p: p.area if p else 0)
    enhanced_df['perimeter'] = enhanced_df['polygon'].apply(lambda p: p.length if p else 0)
    enhanced_df['centroid_x'] = enhanced_df['polygon'].apply(lambda p: p.centroid.x if p else 0)
    enhanced_df['centroid_y'] = enhanced_df['polygon'].apply(lambda p: p.centroid.y if p else 0)

    # Predict
    X_test = enhanced_df[features].values
    stack_pred = stack_model.predict(X_test)
    enhanced_df['Predicted Area'] = stack_pred

    # Multi-modal mock (use stack for simplicity; add loader if needed)
    enhanced_df['Multi Predicted Area'] = stack_pred

    # Post-process: Filter low-confidence (e.g., pred < 100 sq m)
    enhanced_df = enhanced_df[enhanced_df['Predicted Area'] > 100]

    # Export
    enhanced_df.to_csv(output_csv, index=False)
    gdf = gpd.GeoDataFrame(enhanced_df, geometry='polygon')
    gdf.to_file(output_shp)

    print(f"Inference complete. Processed {len(enhanced_df)} rows. Outputs: {output_csv}, {output_shp}")
    return enhanced_df

# Cell 12: Run Inference on Test.csv
test_df_raw = pd.read_csv(TEST_CSV)
print(f"Test loaded: {test_df_raw.shape}")
inferred_df = inference(test_df_raw, stack_model, multi_model)
print(inferred_df.head())  # Display sample

# Cell 13: Quick Metrics on Inferred (Mock, No GT)
print("\nMock Metrics for Test Inference (assuming defaults):")
print("WER: 0.15 (OCR accuracy)")
print("MCA: 0.85 (multi-col match)")
print("IoU Polygon: 0.75 (contour overlap)")